In [ ]:
!pip install haversine

In [ ]:
# Note: From ChatGPT-o1
import networkx as nx
import time
import signal
import psutil
from haversine import haversine
import heapq

# Create the graph
G = nx.Graph()

# Add nodes with coordinates (latitude, longitude)
G.add_node('New York', pos=(40.7128, -74.0060))
G.add_node('Los Angeles', pos=(34.0522, -118.2437))
G.add_node('Chicago', pos=(41.8781, -87.6298))
G.add_node('Houston', pos=(29.7604, -95.3698))
G.add_node('Phoenix', pos=(33.4484, -112.0740))
# Add more cities as needed...

# Add edges (unweighted and bidirectional)
G.add_edge('New York', 'Chicago')
G.add_edge('Chicago', 'Houston')
G.add_edge('Houston', 'Los Angeles')
G.add_edge('Los Angeles', 'Phoenix')
G.add_edge('Phoenix', 'New York')
# Add more connections as needed...

# Timeout handler
def timeout_handler(signum, frame):
    raise TimeoutError

signal.signal(signal.SIGALRM, timeout_handler)

# Function to calculate total distance of a path
def calculate_total_distance(path):
    total_distance = 0
    for i in range(len(path) - 1):
        coord1 = G.nodes[path[i]]['pos']
        coord2 = G.nodes[path[i + 1]]['pos']
        total_distance += haversine(coord1, coord2)
    return total_distance

# Heuristic function for A* and Best-First Search
def heuristic(node, goal):
    coord1 = G.nodes[node]['pos']
    coord2 = G.nodes[goal]['pos']
    return haversine(coord1, coord2)

# Breadth-First Search
def bfs_search(start, goal, timeout_duration=10):
    signal.alarm(timeout_duration)
    try:
        start_time = time.time()
        process = psutil.Process()

        path = nx.shortest_path(G, source=start, target=goal)

        end_time = time.time()
        total_time = end_time - start_time
        total_distance = calculate_total_distance(path)
        memory_usage = process.memory_info().rss / (1024 * 1024)

        print(f"BFS Path: {path}")
        print(f"Total Time: {total_time:.6f} seconds")
        print(f"Total Distance: {total_distance:.2f} km")
        print(f"Memory Usage: {memory_usage:.2f} MB")

        signal.alarm(0)
    except TimeoutError:
        print("BFS Search timed out")

# Depth-First Search
def dfs_search(start, goal, timeout_duration=10):
    signal.alarm(timeout_duration)
    try:
        start_time = time.time()
        process = psutil.Process()

        path = dfs_path(G, start, goal)
        if path is None:
            print("DFS Search failed to find a path")
            return

        end_time = time.time()
        total_time = end_time - start_time
        total_distance = calculate_total_distance(path)
        memory_usage = process.memory_info().rss / (1024 * 1024)

        print(f"DFS Path: {path}")
        print(f"Total Time: {total_time:.6f} seconds")
        print(f"Total Distance: {total_distance:.2f} km")
        print(f"Memory Usage: {memory_usage:.2f} MB")

        signal.alarm(0)
    except TimeoutError:
        print("DFS Search timed out")

def dfs_path(graph, start, goal):
    stack = [(start, [start])]
    visited = set()
    while stack:
        (vertex, path) = stack.pop()
        if vertex == goal:
            return path
        if vertex not in visited:
            visited.add(vertex)
            for neighbor in graph.neighbors(vertex):
                if neighbor not in visited:
                    stack.append((neighbor, path + [neighbor]))
    return None

# Iterative Deepening Depth-First Search
def iddfs_search(start, goal, timeout_duration=10, max_depth=1000):
    signal.alarm(timeout_duration)
    try:
        start_time = time.time()
        process = psutil.Process()

        for depth in range(max_depth):
            path = dls(G, start, goal, depth)
            if path is not None:
                end_time = time.time()
                total_time = end_time - start_time
                total_distance = calculate_total_distance(path)
                memory_usage = process.memory_info().rss / (1024 * 1024)

                print(f"IDDFS Path: {path}")
                print(f"Depth: {depth}")
                print(f"Total Time: {total_time:.6f} seconds")
                print(f"Total Distance: {total_distance:.2f} km")
                print(f"Memory Usage: {memory_usage:.2f} MB")

                signal.alarm(0)
                return
        print("IDDFS Search failed to find a path within max depth")
    except TimeoutError:
        print("IDDFS Search timed out")

def dls(graph, start, goal, depth):
    return recursive_dls(graph, start, goal, depth, path=[start], visited=set())

def recursive_dls(graph, node, goal, depth, path, visited):
    if depth == 0 and node == goal:
        return path
    elif depth > 0:
        visited.add(node)
        for neighbor in graph.neighbors(node):
            if neighbor not in visited:
                result = recursive_dls(graph, neighbor, goal, depth - 1, path + [neighbor], visited)
                if result is not None:
                    return result
        visited.remove(node)
    return None

# Best-First Search
def best_first_search(start, goal, timeout_duration=10):
    signal.alarm(timeout_duration)
    try:
        start_time = time.time()
        process = psutil.Process()

        frontier = []
        heapq.heappush(frontier, (heuristic(start, goal), [start]))
        explored = set()

        while frontier:
            (est_cost, path) = heapq.heappop(frontier)
            node = path[-1]
            if node == goal:
                end_time = time.time()
                total_time = end_time - start_time
                total_distance = calculate_total_distance(path)
                memory_usage = process.memory_info().rss / (1024 * 1024)

                print(f"Best-First Path: {path}")
                print(f"Total Time: {total_time:.6f} seconds")
                print(f"Total Distance: {total_distance:.2f} km")
                print(f"Memory Usage: {memory_usage:.2f} MB")

                signal.alarm(0)
                return
            explored.add(node)
            for neighbor in G.neighbors(node):
                if neighbor not in explored:
                    total_path = path + [neighbor]
                    heapq.heappush(frontier, (heuristic(neighbor, goal), total_path))
        print("Best-First Search failed to find a path")
    except TimeoutError:
        print("Best-First Search timed out")

# A* Search
def a_star_search(start, goal, timeout_duration=10):
    signal.alarm(timeout_duration)
    try:
        start_time = time.time()
        process = psutil.Process()

        path = nx.astar_path(G, start, goal, heuristic=heuristic, weight=lambda u, v, d: 1)

        end_time = time.time()
        total_time = end_time - start_time
        total_distance = calculate_total_distance(path)
        memory_usage = process.memory_info().rss / (1024 * 1024)

        print(f"A* Path: {path}")
        print(f"Total Time: {total_time:.6f} seconds")
        print(f"Total Distance: {total_distance:.2f} km")
        print(f"Memory Usage: {memory_usage:.2f} MB")

        signal.alarm(0)
    except TimeoutError:
        print("A* Search timed out")
    except nx.NetworkXNoPath:
        print("No path found by A* Search")

# Example usage
start_city = 'New York'
goal_city = 'Los Angeles'

print("Breadth-First Search:")
bfs_search(start_city, goal_city)

print("\nDepth-First Search:")
dfs_search(start_city, goal_city)

print("\nIterative Deepening Depth-First Search:")
iddfs_search(start_city, goal_city)

print("\nBest-First Search:")
best_first_search(start_city, goal_city)

print("\nA* Search:")
a_star_search(start_city, goal_city)


In [ ]:
from collections import deque

# BFS from given source s
def bfs(adj, s):

    # Create a queue for BFS
    q = deque()

    # Initially mark all the vertices as not visited
    # When we push a vertex into the q, we mark it as
    # visited
    visited = [False] * len(adj);

    # Mark the source node as visited and enqueue it
    visited[s] = True
    q.append(s)

    # Iterate over the queue
    while q:

        # Dequeue a vertex from queue and print it
        curr = q.popleft()
        print(curr, end=" ")

        # Get all adjacent vertices of the dequeued
        # vertex. If an adjacent has not been visited,
        # mark it visited and enqueue it
        for x in adj[curr]:
            if not visited[x]:
                visited[x] = True
                q.append(x)

# Function to add an edge to the graph
def add_edge(adj, u, v):
    adj[u].append(v)
    adj[v].append(u)

# Example usage
if __name__ == "__main__":

    # Number of vertices in the graph
    V = 5

    # Adjacency list representation of the graph
    adj = [[] for _ in range(V)]

    # Add edges to the graph
    add_edge(adj, 0, 1)
    add_edge(adj, 0, 2)
    add_edge(adj, 1, 3)
    add_edge(adj, 1, 4)
    add_edge(adj, 2, 4)

    # Perform BFS traversal starting from vertex 0
    print("BFS starting from 0: ")
    bfs(adj, 0)


In [ ]:
# Python program to print DFS traversal from a given
# given graph
from collections import defaultdict

# This class represents a directed graph using adjacency
# list representation
class Graph:

    def __init__(self,vertices):

        # No. of vertices
        self.V = vertices

        # default dictionary to store graph
        self.graph = defaultdict(list)

    # function to add an edge to graph
    def addEdge(self,u,v):
        self.graph[u].append(v)

    # A function to perform a Depth-Limited search
    # from given source 'src'
    def DLS(self,src,target,maxDepth):

        if src == target : return True

        # If reached the maximum depth, stop recursing.
        if maxDepth <= 0 : return False

        # Recur for all the vertices adjacent to this vertex
        for i in self.graph[src]:
                if(self.DLS(i,target,maxDepth-1)):
                    return True
        return False

    # IDDFS to search if target is reachable from v.
    # It uses recursive DLS()
    def IDDFS(self,src, target, maxDepth):

        # Repeatedly depth-limit search till the
        # maximum depth
        for i in range(maxDepth):
            if (self.DLS(src, target, i)):
                return True
        return False

# Create a graph given in the above diagram
g = Graph (7);
g.addEdge(0, 1)
g.addEdge(0, 2)
g.addEdge(1, 3)
g.addEdge(1, 4)
g.addEdge(2, 5)
g.addEdge(2, 6)

target = 6; maxDepth = 3; src = 0

if g.IDDFS(src, target, maxDepth) == True:
    print ("Target is reachable from source " +
        "within max depth")
else :
    print ("Target is NOT reachable from source " +
        "within max depth")

# This code is contributed by Neelam Pandey


In [ ]:
from queue import PriorityQueue
v = 14
graph = [[] for i in range(v)]

# Function For Implementing Best First Search
# Gives output path having lowest cost


def best_first_search(actual_Src, target, n):
	visited = [False] * n
	pq = PriorityQueue()
	pq.put((0, actual_Src))
	visited[actual_Src] = True

	while pq.empty() == False:
		u = pq.get()[1]
		# Displaying the path having lowest cost
		print(u, end=" ")
		if u == target:
			break

		for v, c in graph[u]:
			if visited[v] == False:
				visited[v] = True
				pq.put((c, v))
	print()

# Function for adding edges to graph


def addedge(x, y, cost):
	graph[x].append((y, cost))
	graph[y].append((x, cost))


# The nodes shown in above example(by alphabets) are
# implemented using integers addedge(x,y,cost);
addedge(0, 1, 3)
addedge(0, 2, 6)
addedge(0, 3, 5)
addedge(1, 4, 9)
addedge(1, 5, 8)
addedge(2, 6, 12)
addedge(2, 7, 14)
addedge(3, 8, 7)
addedge(8, 9, 5)
addedge(8, 10, 6)
addedge(9, 11, 1)
addedge(9, 12, 10)
addedge(9, 13, 2)

source = 0
target = 9
best_first_search(source, target, v)

# This code is contributed by Jyotheeswar Ganne


In [ ]:
import math
import heapq

# Define the Cell class
class Cell:
    def __init__(self):
        self.parent_i = 0  # Parent cell's row index
        self.parent_j = 0  # Parent cell's column index
        self.f = float('inf')  # Total cost of the cell (g + h)
        self.g = float('inf')  # Cost from start to this cell
        self.h = 0  # Heuristic cost from this cell to destination

# Define the size of the grid
ROW = 9
COL = 10

# Check if a cell is valid (within the grid)
def is_valid(row, col):
    return (row >= 0) and (row < ROW) and (col >= 0) and (col < COL)

# Check if a cell is unblocked
def is_unblocked(grid, row, col):
    return grid[row][col] == 1

# Check if a cell is the destination
def is_destination(row, col, dest):
    return row == dest[0] and col == dest[1]

# Calculate the heuristic value of a cell (Euclidean distance to destination)
def calculate_h_value(row, col, dest):
    return ((row - dest[0]) ** 2 + (col - dest[1]) ** 2) ** 0.5

# Trace the path from source to destination
def trace_path(cell_details, dest):
    print("The Path is ")
    path = []
    row = dest[0]
    col = dest[1]

    # Trace the path from destination to source using parent cells
    while not (cell_details[row][col].parent_i == row and cell_details[row][col].parent_j == col):
        path.append((row, col))
        temp_row = cell_details[row][col].parent_i
        temp_col = cell_details[row][col].parent_j
        row = temp_row
        col = temp_col

    # Add the source cell to the path
    path.append((row, col))
    # Reverse the path to get the path from source to destination
    path.reverse()

    # Print the path
    for i in path:
        print("->", i, end=" ")
    print()

# Implement the A* search algorithm
def a_star_search(grid, src, dest):
    # Check if the source and destination are valid
    if not is_valid(src[0], src[1]) or not is_valid(dest[0], dest[1]):
        print("Source or destination is invalid")
        return

    # Check if the source and destination are unblocked
    if not is_unblocked(grid, src[0], src[1]) or not is_unblocked(grid, dest[0], dest[1]):
        print("Source or the destination is blocked")
        return

    # Check if we are already at the destination
    if is_destination(src[0], src[1], dest):
        print("We are already at the destination")
        return

    # Initialize the closed list (visited cells)
    closed_list = [[False for _ in range(COL)] for _ in range(ROW)]
    # Initialize the details of each cell
    cell_details = [[Cell() for _ in range(COL)] for _ in range(ROW)]

    # Initialize the start cell details
    i = src[0]
    j = src[1]
    cell_details[i][j].f = 0
    cell_details[i][j].g = 0
    cell_details[i][j].h = 0
    cell_details[i][j].parent_i = i
    cell_details[i][j].parent_j = j

    # Initialize the open list (cells to be visited) with the start cell
    open_list = []
    heapq.heappush(open_list, (0.0, i, j))

    # Initialize the flag for whether destination is found
    found_dest = False

    # Main loop of A* search algorithm
    while len(open_list) > 0:
        # Pop the cell with the smallest f value from the open list
        p = heapq.heappop(open_list)

        # Mark the cell as visited
        i = p[1]
        j = p[2]
        closed_list[i][j] = True

        # For each direction, check the successors
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
        for dir in directions:
            new_i = i + dir[0]
            new_j = j + dir[1]

            # If the successor is valid, unblocked, and not visited
            if is_valid(new_i, new_j) and is_unblocked(grid, new_i, new_j) and not closed_list[new_i][new_j]:
                # If the successor is the destination
                if is_destination(new_i, new_j, dest):
                    # Set the parent of the destination cell
                    cell_details[new_i][new_j].parent_i = i
                    cell_details[new_i][new_j].parent_j = j
                    print("The destination cell is found")
                    # Trace and print the path from source to destination
                    trace_path(cell_details, dest)
                    found_dest = True
                    return
                else:
                    # Calculate the new f, g, and h values
                    g_new = cell_details[i][j].g + 1.0
                    h_new = calculate_h_value(new_i, new_j, dest)
                    f_new = g_new + h_new

                    # If the cell is not in the open list or the new f value is smaller
                    if cell_details[new_i][new_j].f == float('inf') or cell_details[new_i][new_j].f > f_new:
                        # Add the cell to the open list
                        heapq.heappush(open_list, (f_new, new_i, new_j))
                        # Update the cell details
                        cell_details[new_i][new_j].f = f_new
                        cell_details[new_i][new_j].g = g_new
                        cell_details[new_i][new_j].h = h_new
                        cell_details[new_i][new_j].parent_i = i
                        cell_details[new_i][new_j].parent_j = j

    # If the destination is not found after visiting all cells
    if not found_dest:
        print("Failed to find the destination cell")

def main():
    # Define the grid (1 for unblocked, 0 for blocked)
    grid = [
        [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
        [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
        [1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
        [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 1]
    ]

    # Define the source and destination
    src = [8, 0]
    dest = [0, 0]

    # Run the A* search algorithm
    a_star_search(grid, src, dest)

if __name__ == "__main__":
    main()
